# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [10]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [11]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [13]:
browser = webdriver.Chrome('C://Users/Lenovo/chromedriver.exe')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # 模擬網頁下拉
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 等待一段時間讓網頁更新
    time.sleep(SCROLL_PAUSE_TIME)

    # 檢視目前網頁底端
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height
    print("網頁更新中...")

網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [14]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [ ]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)

----------------------------------------------------------------------

[0] 銅板價買進指數ETN　搶進台股反彈機會

▲ 投資人可以趁低檔布局台股ETN。（示意圖／pixabay）
記者李瑞瑾／綜合報導
加權指數開紅盤後大幅度從高點回落，台股ETN也隨之回檔，統一證券金融商品部表示，目前市場氣氛趨於緩和樂觀，投資人若看好後市，用指數ETN參與不但可以省去選股的風險與麻煩，重點是用低單價買進指數，可以更有效率參與市場回溫。
統一證券表示，目前市場上連結台股的ETN，其指數成分股皆為各產業的龍頭股，也是領漲指標，去年指數位於高點時投資人入場有疑慮，目前回檔可當作一籃子好股票打折，善用單價相對低且無追蹤誤差的ETN入場不但無負擔，也不用費心去賭哪一個類股哪一檔會先反彈。
以漲升股利150報酬指數(IR0091)為例，去年5月大盤重挫後反彈，兩個月漲幅在8%以上，8月指數再度回檔，反彈後至12月漲幅接近15%，若是用該ETN去年4/30掛牌以後逢低點便加碼買進1張計算，至今年2/6報酬率達9%以上。而不耐震盪的投資人，則可以選擇抗震且波動低的指數。
統一證券金融商品部建議，投資人若想趁低檔入市，可以逢低點分批進場，用便宜的價格建立短線抗跌跟漲，同時也具備高殖利率特性的指數部位，用不到兩萬元的小錢真正參與大行情。
----------------------------------------------------------------------

[1] 汽油價格明起不調整！　柴油價格調降0.1元

記者姚惠茹／台北報導
台灣中油今（16）日宣布，自明（17）日凌晨零時起汽油價格不調整，柴油價格每公升調降0.1元，調整後的參考零售價格分別為92無鉛汽油每公升24.9元、95無鉛汽油每公升26.4元、98無鉛汽油每公升28.4元、超級柴油每公升22.6元。
▲汽油價格明起不調整、柴油價格調降0.1元。（圖／記者張一中攝）
台灣中油表示，受新冠肺炎（COVID-19）疫情引發原油期貨市場恐慌性賣壓因素影響，本週國際原油價格續跌。本週調價指標7D3B週均價為每桶54.39美元，較前週54.32美元上漲0.07美元，新臺幣兌美元匯率升值0.122元，國內油價依公式計算跌幅為0.22％。
台灣中油指出，

▲三星隨身充無線版。（圖／取自9TO5Google）
記者謝仁傑／綜合報導
這個無線隨身充表面有一個充電板，採用雙USB-C插孔，不需要攜帶充電線，也可以隨時將手機放在上面充電，售價79.99美元（約新台幣2,404元）。
----------------------------------------------------------------------

[12] 期交所證實董事長許虞哲今日下午病逝　享壽67歲

▲前財政部長、現任期貨交易所董事長許虞哲過世。（圖／資料照）
記者余弦妙／台北報導
前財政部長、現任期貨交易所董事長許虞哲日前因病入院治療，今天下午傳出過世消息，享壽67歲。
而期交所表示，臺灣期貨交易所董事長許虞哲先生於109年2月15日下午病逝。全體同仁深表不捨，有關許董事長治喪事宜，將與家屬聯繫後，對外說明。
據了解，許虞哲去年底原本預計會出席歲末記者聯誼餐會，不料最後因身體不適取消出席，由期交所總經理黃炳鈞代為主持，如今卻傳出許虞哲因病不治消息，不禁令人遺憾。
----------------------------------------------------------------------

[13] 快訊／前財政部長許虞哲過世　享壽67歲

記者林彥臣／綜合報導
前財政部長、現任期貨交易所董事長許虞哲，驚傳在今天（15日）下午過世，享壽67歲。許虞哲於2016年5月20日起，出任林全內閣的財政部長，2018年內閣改組後，才轉任期交所董事長。
根據《中央社》報導，友人透露許虞哲日前因病住院治療，在15日下午過世。
許虞哲1952年出生，政治大學財政研究所畢業後，赴美深造取得哈佛大學法學碩士學位，擁稅務、法務雙重專長，從財政部賦稅署基層做起，歷經台北市稅捐稽徵處處長、五區國稅局長，後來擔任賦稅署長、財政部常務次長，2016年5月20日出任財政部長，公職生涯幾乎都奉獻給財政部，2018年7月內閣改組，才轉任期交所董事長。
►偷偷分享少女秘密！
----------------------------------------------------------------------

[14] 快訊／期交所董事長許虞哲逝世　享壽67歲！公職生涯奉獻財政部

中央社訊
前財政部長、現任期貨交易所董事長許虞哲日前因病入院治療，根據友人透

▲Android系統又爆出資安危機。（圖／翻攝自pixabay）
記者陳心怡／外電報導
Android手機安全問題漏洞百出，先前Google Play Store有多款惡意應用程式，而現在近日又有資安機構《Cofense》揭露了新的Android系統木馬病毒「Anubis」。
但這款病毒不同於以往夾藏在App中，而是以釣魚郵件的方式，或偽裝成銀行收據之類的文件檔案進行攻擊，當用戶不慎開啟後，就會關閉 Google Play Protect 防護功能，然後開啟該病毒需要的權限。
以駭客所列出的逾250個攻擊目標進行比對，「Anubis」大部分會鎖定金融、銀行和網購店商平台為主的APP，當用戶打開這些 APP時，病毒就會製作一個假的登錄頁面讓用戶登入，藉此竊取用戶的使用者憑證。
Anubis的能力不僅於此，它還會透過用戶的截圖、系統設定、開啟任何網頁、語音紀錄、 通話、GPS位置等等，將所有資料傳送到駭客的伺服器。
對此，「Cofense」呼籲用戶應該要確認自身所使用的軟體，是否來自有公信力的開發者，同時也建議企業應限制員工裝置上安裝的程式，確保在辦公環境使用的程式來自可靠的開發者，降低被惡意病毒攻擊的危險。
----------------------------------------------------------------------

[25] 影／機器犬也能拉車！　快來看《流言終結者》主持人如何應用

▲波士頓動力公司(Boston Dynamics)開發的Spot機器犬。（圖／路透社）
記者謝仁傑／綜合報導
Savage安裝一個可以連接到人力車的牽引鉤，以此來跟機器犬做連結。他還幽默指出，機器犬能承受的重量有限，人直接騎在它上面是不可能的，但用來拉人力車是一個很好的替代品。
影片中，Adam Savage坐在人力車上，只見機器犬相當輕鬆地拉動人力車，讓 Savage笑聲不斷，直呼：「太驚人了！」，他還為機器犬加油打氣，開玩笑地說「你可以的」。
機器犬是由類似無人機的掌中控制器，以無線藍牙方式來操控。
此前，警方還曾將Spot作為探索危險的機械警犬，如今Spot可能會有越來越多層面的用途，甚至還有人認為有發展成軍武的可能。
不過開發Spot的波士頓動力公司的副總裁Michael Perry曾堅決表示，公司沒有興趣將Spot發展成軍武，也已將這個理念列入

▲家登董事長邱銘乾。（圖／資料照）
記者周康玉／台北報導
EUV光罩盒廠家登精密（3680）於今（15）日公布1月營收為1.43億元，月減48%、年減19%，因一月份工作天數因傳統農曆年假大幅減少、僅17天，導致客戶訂單及出貨時程延後，但受惠於中國半導體客戶轉單效應，2月份半導體本業營收表現可期，第一季會比去年好。
目前武漢疫情影響，尤其中國全面延後開工，當地半導體供應鏈受到衝擊，家登半導體生產基地位於台灣，產能及出貨狀況均不受影響。此外，家登吳江新創汽車公司為吳江當地示範工廠，於2月11日取得開工許可，然疫情連帶影響買氣，短期汽車產業營收將受影響，然而，半導體營收可補足汽車營收缺口，不影響年度整體獲利。
受惠於5G及晶圓代工需求直線上升，7奈米以下先進製程熱度高居不下，去年底到今年全球各大客戶皆陸續上調資本支出，顯示半導體在高階製程持續投入的長期發展趨勢。
家登指出，今年仍是半導體蓬勃發展的一年，市場預估全球半導體產業規模將較去年成長8％，晶圓代工市場規模較去年成長17％；雖然進入傳統淡季的氛圍，半導體的成長力道尚未顯著，然去年底到今年全球資本支出已突破紀錄來到歷史高點，家登將因此受惠，全年營收創高可期。
其中，因艾斯摩爾（ASML）EUV機台供不應求，家登連帶受益，上半年家登位於台南的樹谷廠如火如荼進行新生產設備的安裝及新產線的擴建，除了大坪數無塵室的興建，檢測設備更是重點投資項目，產量提升的同時，嚴格進行品質控管。
----------------------------------------------------------------------

[35] 9部口罩新機台今交機　經濟部力拚下周產能達460萬片

▲經濟部表示，針對國內66家有生產事實的業者口罩產量，已全數徵用。（圖／記者湯興漢攝）
記者林淑慧／台北報導
新冠肺炎疫情尚未平息，經濟部斥資2億元新建口罩產線，首批9部機台今（15）日交機，官員指出，國內口罩本周日產能已突破420萬片，新機台上線後，下周將力拚460萬片日產能。
經濟部工業局表示，口罩新產線的第一批機台今天交機，今天將先進行機組測試，再併入生產行列，預估每部機台每天能生產10萬片，九部機台共90萬片，等到三月新產線機組全數到位，可望推升日產能達千萬片。
新冠肺炎疫情持續延燒，口罩一罩難求，經濟部表示，口罩徵用係由衛福部提供取